In [9]:
import os

def get_pdf_paths(directory):
    pdf_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.pdf'):
                pdf_paths.append(os.path.join(root, file))
    return pdf_paths

# Specify the main folder
main_folder = 'CRIME-DATASETS'

# Get all PDF paths
pdf_paths = get_pdf_paths(main_folder)

# Print all PDF paths
for path in pdf_paths:
    print(path)


CRIME-DATASETS\corruption\corruption1.pdf
CRIME-DATASETS\corruption\corruption2.pdf
CRIME-DATASETS\corruption\corruption3.pdf
CRIME-DATASETS\corruption\corruption4.pdf
CRIME-DATASETS\corruption\corruption5.pdf
CRIME-DATASETS\corruption\Penal_code.pdf
CRIME-DATASETS\divorce\divorce1.pdf
CRIME-DATASETS\divorce\divorce2.pdf
CRIME-DATASETS\divorce\divorce3.pdf
CRIME-DATASETS\divorce\divorce4.pdf
CRIME-DATASETS\divorce\divorce5.pdf
CRIME-DATASETS\divorce\Penal_code.pdf
CRIME-DATASETS\murder\murder1.pdf
CRIME-DATASETS\murder\murder2.pdf
CRIME-DATASETS\murder\murder3.pdf
CRIME-DATASETS\murder\murder4.pdf
CRIME-DATASETS\murder\murder5.pdf
CRIME-DATASETS\murder\Penal_code.pdf
CRIME-DATASETS\property\Penal_code.pdf
CRIME-DATASETS\property\property1.pdf
CRIME-DATASETS\property\property2.pdf
CRIME-DATASETS\property\property3.pdf
CRIME-DATASETS\property\property4.pdf
CRIME-DATASETS\property\property5.pdf
CRIME-DATASETS\rape\Penal_code.pdf
CRIME-DATASETS\rape\rape1.pdf
CRIME-DATASETS\rape\rape2.pdf


In [5]:
import docx2txt as docx2txt
import fitz
import os
import sys

In [6]:
class TextExtractor:
    def __init__(self):
        pass
    #
    def extract_text_from_docx(self, docx_path):
        try:
            text = docx2txt.process(docx_path)
            return text
        except Exception as e:
            return docx_path


    def extract_text_from_pdf(self, pdf_path):
        text = ""
        pdf_document = fitz.open(pdf_path)
        for page_num in range(pdf_document.page_count):
            page = pdf_document.load_page(page_num)
            text += page.get_text()
        pdf_document.close()
        return text

    def rename_doc_to_docx(self, folder_path):
        for filename in os.listdir(folder_path):
            if filename.endswith(".doc"):
                new_filename = os.path.splitext(filename)[0] + ".docx"
                os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_filename))

    def extract_file_names(self, directory_path):
        self.rename_doc_to_docx(directory_path)
        file_names = []
        for root, dirs, files in os.walk(directory_path):
            for file in files:
                file_names.append(file)
        return file_names


class ResumeTextExtractor(TextExtractor):
    def __init__(self):
        super().__init__()

    def extract_text_from_resumes(self, folder_path):
        file_names = self.extract_file_names(folder_path)
        text_dict = {}
        corrupted_files = []

        try:
            for file_name in file_names:
                if file_name.endswith('.pdf'):
                    text = self.extract_text_from_pdf(folder_path + file_name)
                    text_dict[file_name] = text
                elif file_name.endswith('.docx'):
                    text = self.extract_text_from_docx(folder_path + file_name)
                    if text == folder_path + file_name:
                        corrupted_files.append(text)
                    else:
                        text_dict[file_name] = text
            return text,text_dict, corrupted_files

        except Exception as e:
            return None, []


## Extarcting the conetxt form the pdfs

In [10]:
ResumeTextExtractor_ = ResumeTextExtractor()
list_of_dir = ["corruption", "divorce", "murder", "property", "rape", "Robbery", "sexual assault"]

In [28]:
context_text_list = {}

for dir in list_of_dir:
    text ,text_dict, corrupted_files = ResumeTextExtractor_.extract_text_from_resumes(folder_path=f'CRIME-DATASETS/{dir}/')
    for i,text in enumerate(list(text_dict.values())):
        context_text_list[f"{dir}_{i}"] = text


In [30]:
import pandas as pd
sub = pd.DataFrame({

    'context' :  list(context_text_list.values()),
    "case_type" : list(context_text_list.keys())

})

sub.to_csv("submission.csv", index=False)